                                                                                        Nora Hafidi López 
                                                                                        Lucía Inés Merlo

In [1]:
!spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 16.8MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172936 sha256=893c75cda5186cac9ec533027dc0d67a7557db8ae2e371e5a3eb2bb6ff19a309
  Stored in directory: /tmp/pip-ephem-wheel-cache-qi4rz700/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [1]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [2]:
# Procesamiento texto

#pip install TextBlob
#pip install spacy
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
nltk.download('')
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk import SnowballStemmer
from textblob import TextBlob
import string



# Modelos
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

# Métricas
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading : Package '' not found in index


## Librerías

## Carga de los datos

In [4]:
df = pd.read_csv("train 2.csv") 

In [ ]:
df.loc[:,'comment'].head(10)

0                             Pensó: Zumo para restar.
1     Como les gusta el afeitado en seco a esta gente.
2    asi me gusta, que se maten entre ellos y en al...
3    Loss mas valientes, los que mejor cortan nuest...
4                                        Costumbres...
5    lastima que no se volvio loco del todo y se su...
6    los mejores mas preparados para cortar calabac...
7                                   Los más preparados
8    hombre preparado viene para currar en un matad...
9            Los detuvieron en ronda malaga, un saludo
Name: comment, dtype: object

In [ ]:
df['preprocesado'] = df['comment']

In [ ]:
a = df['preprocesado']
a[3] = '1234'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.head()

,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,target_person,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level,preprocesado
0,CR,0_000,0_002,0_002,1,Pensó: Zumo para restar.,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1234
1,CR,0_001,0_003,0_003,1,Como les gusta el afeitado en seco a esta gente.,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,aaa
2,CR,0_002,0_004,0_004,1,"asi me gusta, que se maten entre ellos y en al...",0,0,0,0,0,1,0,0,0,0,0,1,1,1,2,ir ser malo
3,CR,0_003,0_005,0_005,1,"Loss mas valientes, los que mejor cortan nuest...",0,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1234
4,CR,0_004,0_006,0_006,1,Costumbres...,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,venir ser sub pole guay


## Procesamiento de texto

In [5]:
replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )

Para la adecuada normalización del texto hemos realizado los siguientes pasos:

1. Eliminar signos de puntuación y los caracteres que no fuesen letras
2. Deshacernos de las stopwords
3. Obtener la raíz de las palabras
4. Quedarnos con el lema
5. Extracción de características 

#### Eliminación de signos de puntuación y ruido ####
Para eliminar los signos de puntuación se ha utilizado una función de la librería string. Esta nos incluye en una lista todos los signos de puntuación existentes y basta con recorrer los tokens e ir eliminando los caracteres presentes en la lista. Para la eliminación de los caracteres no alphanumeric se ha utilizado la función isalpha().

#### Stopwords ####
Con nltk se han eliminado todos las stopwords presentes en el lenguaje español. 

#### Raíz de las palabras ####
Con la función PorterStemmer() se dispone a sustituir cada palabra por su raíz. Por lo tanto, palabras con la misma raíz, tendrán un significado similar.  

#### Lema ####
Ahora, se procede a lematizar el texo con lemma_.lower()

#### Extracción de características ####
Para esto, se utilizan bag of words

In [8]:
errores = []
lemmas = []
#a = df['preprocesado']
for i in range(len(df)):
        #tokenizar
        tokens = nltk.word_tokenize(df.loc[:,'comment'].iloc[i])
        #elimino signos de puntuación y ruido
        #table = str.maketrans('', '', string.punctuation)
        tokens = ' '.join(tokens)
        tokens = tokens.lower()
        punct = string.punctuation
        for c in punct:
            tokens = tokens.replace(c, " ")
        for a1, b1 in replacements:
            tokens = tokens.replace(a1, b1).replace(a1.upper(), b1.upper())
            stripped = tokens.split()
        #stripped = [w.translate(table) for w in tokens] #intento sacar los signos de puntuación pero no puedo
        #stripped = re.sub('[%s]' % re.escape(string.punctuation), ' ', tokens)

        words = [word for word in stripped if word.isalpha()]


        #eliminar stopwords
        stop_words = set(stopwords.words('spanish'))
        #words = [w for w in stripped if not w in stop_words]
        words = [word for word in stripped if not word in stop_words]

        #stemming
        porter = PorterStemmer()
        for i in range(len(words)):
            words[i] = porter.stem(words[i])

        #stemmed = [porter.stem(word) for word in words]

        #lemmatizar 
        words = ' '.join(words)
        doc = nlp(words)
        lemmas1 = [tok.lemma_.lower() for tok in doc]
        b = ' '.join(lemmas1)
        lemmas.append(b)
        #a[i] = b
lemmas

#extraction
try:
    vectorizer = CountVectorizer()
    vectorizer.fit(lemmas)
    X_bag_of_words = vectorizer.transform(lemmas)
except ValueError:
    errores.append(lemmas1)

In [ ]:
for i in range(len(lemmas)):
  a[i] = lemmas[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#df.head(100)
import csv
df.to_csv('train_prep.csv', sep = ',')

In [ ]:
#print(X_bag_of_words)

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(lemmas)
tfidf = tfidf_vectorizer.transform(lemmas)
#print(tfidf)

## Detección de toxicidad

### Despliegue de modelos, evaluación con f1-score y CEM

In [ ]:
y = df['toxicity']
X = tfidf
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, 
                                                    random_state = 0)

#### Árboles de decisión

In [ ]:
clf_dt = tree.DecisionTreeClassifier(random_state = 0, class_weight="balanced",
                                     max_depth = 6)
clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_dt))

scores_dt = cross_val_score(clf_dt, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation: ", scores_dt.mean())

scores_dt = cross_val_score(clf_dt, X_test, y_test, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation del test: ", scores_dt.mean())

print(classification_report(y_test,y_pred_dt))

Accuracy del modelo: 0.6854256854256854
Media de f1-score con cross_validation:  0.5521910321268637
Media de f1-score con cross_validation del test:  0.4662880217178749
              precision    recall  f1-score   support

           0       0.69      0.94      0.80       449
           1       0.67      0.21      0.32       244

    accuracy                           0.69       693
   macro avg       0.68      0.58      0.56       693
weighted avg       0.68      0.69      0.63       693



In [ ]:
#CEM

#### SVM

In [ ]:
clf_svm = svm.SVC(random_state=0,C=1, class_weight="balanced")
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_svm))

scores_svm = cross_val_score(clf_svm, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_svm.mean())

scores_svm = cross_val_score(clf_svm, X_test, y_test, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_svm.mean())
print(classification_report(y_test,y_pred_svm))

Accuracy del modelo: 0.7215007215007215
Media de f1-score con cross_validation 0.629242520791558
Media de f1-score con cross_validation en test:  0.5306070173027698
              precision    recall  f1-score   support

           0       0.73      0.91      0.81       449
           1       0.70      0.37      0.48       244

    accuracy                           0.72       693
   macro avg       0.71      0.64      0.65       693
weighted avg       0.72      0.72      0.69       693



#### Logistic regression

In [ ]:
clf_log = LogisticRegression(random_state = 0,class_weight="balanced")
clf_log.fit(X_train, y_train)
y_pred_log = clf_log.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_log))

scores_log = cross_val_score(clf_log, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_log.mean())

scores_log = cross_val_score(clf_log, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_log.mean())
print(classification_report(y_test,y_pred_log))

Accuracy del modelo: 0.7171717171717171
Media de f1-score con cross_validation 0.6716249668433196
Media de f1-score con cross_validation en test:  0.6210797267076509
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       449
           1       0.62      0.52      0.57       244

    accuracy                           0.72       693
   macro avg       0.69      0.67      0.68       693
weighted avg       0.71      0.72      0.71       693



#### KNN

In [ ]:
# Obtención del número óptimo de vecinos

neighbors = list(range(1,10,2)) 
cv_scores = [ ]

for K in neighbors: #(k=1,k=3,k=5,k=7 y k=9)
    knn = KNeighborsClassifier(n_neighbors = K,weights="distance",metric="euclidean")
    scores = cross_val_score(knn,X_train,y_train,cv = 5,scoring ="accuracy")
    cv_scores.append(scores.mean())

mse = [1-x for x in cv_scores]

optimal_k = neighbors[mse.index(min(mse))] #calculo el minimo error obtenido para k
print("The optimal no. of neighbors is {}".format(optimal_k))

The optimal no. of neighbors is 1


In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors = 1,weights="distance",metric="euclidean")
clf_knn.fit(X_train,y_train)
y_pred_knn = clf_knn.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_knn))
print(classification_report(y_test,y_pred_knn))
scores_knn = cross_val_score(clf_knn, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_knn.mean())
scores_knn = cross_val_score(clf_knn, X_test, y_test, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_knn.mean())

Accuracy del modelo: 0.6594516594516594
              precision    recall  f1-score   support

           0       0.66      0.98      0.79       449
           1       0.68      0.06      0.11       244

    accuracy                           0.66       693
   macro avg       0.67      0.52      0.45       693
weighted avg       0.67      0.66      0.55       693

Media de f1-score con cross_validation 0.47694603914593897
Media de f1-score con cross_validation en test:  0.43776079991626043


### Grid Search

#### Árboles de decisión

In [ ]:
model_dt = tree.DecisionTreeClassifier(class_weight='balanced',random_state = 0)

grid_dt=dict(criterion= ("gini", "entropy"),splitter=("best", "random"),
             max_depth=list(range(1,8)))

clf_dt = GridSearchCV(estimator = model_dt,
                     param_grid = grid_dt)

mod_dt=clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)

In [ ]:
mod_dt.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='random')

In [ ]:
#clf_searc_dt = tree.DecisionTreeClassifier(random_state = 0, class_weight="balanced",
 #                                    max_depth = 7, criterion = "entropy")

clf_searc_dt = tree.DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='random')

clf_searc_dt.fit(X_train, y_train)
y_pred_searc_dt = clf_searc_dt.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_searc_dt))
scores_dt = cross_val_score(clf_searc_dt, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation: ", scores_dt.mean())

scores_dt_test = cross_val_score(clf_searc_dt, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation del test: ", scores_dt_test.mean())
print(classification_report(y_test,y_pred_searc_dt))

Accuracy del modelo: 0.6984126984126984
Media de f1-score con cross_validation:  0.5609680913335696
Media de f1-score con cross_validation del test:  0.4738335374223451
              precision    recall  f1-score   support

           0       0.69      0.99      0.81       449
           1       0.89      0.16      0.28       244

    accuracy                           0.70       693
   macro avg       0.79      0.58      0.54       693
weighted avg       0.76      0.70      0.62       693



#### SVM

In [ ]:
clf_search_svm = GridSearchCV(estimator=svm.SVC(),
                              param_grid={'C': [1, 10],
                                          'kernel': ('linear', 'rbf','poly','sigmoid')
                                         })
m_svm = clf_search_svm.fit(X_train, y_train)
m_svm.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#clf_linear_svm = svm.SVC(kernel='linear',C=1,class_weight="balanced")

clf_linear_svm = svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

clf_linear_svm.fit(X_train, y_train)
y_pred_linear_svm = clf_linear_svm.predict(X_test)

In [ ]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_linear_svm))
print(classification_report(y_test,y_pred_linear_svm))

scores_knn = cross_val_score(clf_linear_svm, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation: ", scores_knn.mean())

scores_knn_test = cross_val_score(clf_linear_svm, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation del test: ", scores_knn_test.mean())

Accuracy del modelo: 0.7171717171717171
              precision    recall  f1-score   support

           0       0.72      0.93      0.81       449
           1       0.71      0.33      0.45       244

    accuracy                           0.72       693
   macro avg       0.71      0.63      0.63       693
weighted avg       0.72      0.72      0.68       693

Media de f1-score con cross_validation:  0.6253405316778935
Media de f1-score con cross_validation del test:  0.5798695111847556


#### Logistic regression

In [ ]:
from scipy.stats import uniform, norm
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold


model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search

grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid,
                           n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
# summarize results
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
 #   print("%f (%f) con: %r" % (mean, stdev, param))

Mejor: 0.733454 usando {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}


In [ ]:
clf_log_tun = LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced")
clf_log_tun.fit(X_train, y_train)
y_pred_log_tun = clf_log_tun.predict(X_test)

In [ ]:
scores_log_tun = cross_val_score(clf_log_tun, X_train, y_train, cv=5, scoring='f1_macro')
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_log_tun))
print(classification_report(y_test,y_pred_log_tun))
print("Media de f1-score con cross_validation", scores_log_tun.mean())

scores_logistic_tun = cross_val_score(clf_log_tun, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_logistic_tun.mean())

Accuracy del modelo: 0.7128427128427128
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       449
           1       0.61      0.52      0.56       244

    accuracy                           0.71       693
   macro avg       0.68      0.67      0.67       693
weighted avg       0.70      0.71      0.71       693

Media de f1-score con cross_validation 0.6624594010643858
Media de f1-score con cross_validation en test:  0.6299135643129212


## Detección de errores Regresión Logística

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def read_data(link):
	dataset = pd.read_csv(link,sep=',')
	texts = list(dataset['comment'])
	t1_label = list(dataset['toxicity'])
	t2_label = list(dataset['toxicity_level'])
	return texts, t1_label, t2_label

def read_data_exit(link,lang):
	dataset = pd.read_csv(link,sep='\t')
	dataset = dataset[dataset['language'] == lang]
	texts = list(dataset['text'])
	t1_label = list(dataset['task1']) 
	t2_label = list(dataset['task2'])
	return texts, t1_label, t2_label



def error_a(train,y_train):

	X_train, X_test, y_train, y_test = train_test_split(train, y_train, test_size=0.3, random_state=42)

	vectorizer = TfidfVectorizer(ngram_range=(2,2))
	train = vectorizer.fit_transform(X_train)
	test = vectorizer.transform(X_test)
 

  vectorizer = TfidfVectorizer()
  train = tfidf_vectorizer.fit(lemmas)
  test = tfidf_vectorizer.transform(lemmas)

  clf = LogisticRegression(random_state = 0, C=10,penalty='l2' 
                           ,solver = 'liblinear',class_weight="balanced")
  clf.fit(train, y_train)
  preds = clf_log_tun.predict(test)

	ids = [i for i in range(len(y_test))]
	df = pd.DataFrame({'id':ids,'texts':X_test,'target':y_test,'predicted':preds})

	df['error'] = abs(df['target'] - df['predicted']) 

	errors = df[df['error']==1]

	errors['length'] = errors['texts'].apply(lambda x: len(x))
	errors['long'] = errors['texts'].apply(lambda x: len(x)>20 )
	print(errors)

IndentationError: ignored

In [ ]:
texts, t1_label, t2_label = read_data('train.csv')

errors = error_a(texts, t1_label)
errors

        id                                              texts  ...  length   long
3        3                                   En una catapulta  ...      16  False
5        5  Eso es una puta estupidez.\r\n\r\nCon todo tu ...  ...     417   True
7        7  Shur, años de entrenamiento....sabes cuantas c...  ...      87   True
8        8  Vaya, vaya, es tan difícil reconocer el fenoti...  ...     438   True
9        9  De escuela femenina. Para que les enseñe a fon...  ...      77   True
...    ...                                                ...  ...     ...    ...
1031  1031  diso mío. Levitas 20 cm por encima del suelo d...  ...     265   True
1033  1033  . Ahora le pongo el enlace para 50 sombras de ...  ...      72   True
1034  1034  Pues esto si que había que subvencionarlo si a...  ...      59   True
1035  1035        a algunos las neuronas no les dan para mas.  ...      43   True
1037  1037  aqui le damos una paguita al pobre negro de lo...  ...      55   True

[355 rows x 7 c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
import pandas as pd
import numpy as np

df_series = pd.DataFrame(errors)


In [ ]:
df_series.to_csv('errores.csv', sep = '\t')

In [ ]:
print(df_series)

Empty DataFrame
Columns: []
Index: []


#### KNN

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
model = KNeighborsClassifier()
vecinos = list(range(1,10))
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

# define grid search
grid = dict(n_neighbors=vecinos,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, 
                           cv=cv, scoring='accuracy',error_score=0)

grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.691215 using {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}


In [ ]:
clf_searc_knn = KNeighborsClassifier(n_neighbors = 3,
                                     weights="distance",metric="euclidean")

clf_searc_knn.fit(X_train,y_train)
y_pred_searc_knn = clf_searc_knn.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test,y_pred_searc_knn))
print(classification_report(y_test,y_pred_searc_knn))

scores_searc_knn = cross_val_score(clf_searc_knn, X_train, y_train, 
                                   cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_searc_knn.mean())

scores_searc_knn2 = cross_val_score(clf_searc_knn, X_test, y_test, 
                                    cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_searc_knn2.mean())

Accuracy del modelo: 0.6608946608946609
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       449
           1       0.85      0.05      0.09       244

    accuracy                           0.66       693
   macro avg       0.75      0.52      0.44       693
weighted avg       0.72      0.66      0.54       693

Media de f1-score con cross_validation 0.4552436960177454
Media de f1-score con cross_validation en test:  0.42664070107775254


## Solución

Para la detección de toxicidad, la idea es crear modelos cuyos datos de entrada sean tupas de palabras con su respectiva probabilidad de aparición y como salida una variable binomial que indica si el texto es tóxico o no. 
Para ello, con unos datos de entrenamiento, se han probado diferentes modelos: Árboles de decisión, SVM, Regresión Logística, Redes Neuronales y KNN. Luego, en los modelos de máquina de vectores soporte se han modelado los datos con kernel lineal, no lineal, polinomial y sigmoide. Para todos estos modelos, con unos datos de test, se ha calculado la precisión para ver qué tan bien modelan los datos. Cabe mencionar que la precisión es engañosa si los datos no están balanceados por lo que se penaliza los errores de la clase minoritaria con la función class_weight. 
Los resultados han sido los siguientes:

- Árboles de decisión: 0.6637806637806638
- SVM kernel por defecto: 0.35209235209235207
- SVM kernel lineal: 0.7301587301587301
- SVM kernel no lineal: 0.6767676767676768
- SVM kernel polinomial: 0.6652236652236653
- SVM kernel sigmoide: 0.7243867243867244
- Regresión Logística: 0.7171717171717171
- Redes Neuronales: 0.6926406926406926

Los resultados no son del todo buenos, pero de todos los modelos, máquinas de vectores soporte con kernel lineal es el que mejor predice nuestros datos con un 73% de acierto. Sí es cierto que se podrían seeguir mejorando los modelos, pero de momento no está nada mal.  

## Solución del Grid

El grid search en un método que valora todos los posibles hiperparámetros de un modelos y para tus datos de entrada te revela cuáles son los más adecuados. En nuestro caso, hemos realizado el grid search para los mismos modelos que en la sesión anterior, pero obviando NN ya que es demasiado lento y no vale la pena.



Luego de obtener los resultados de los mejores hiperparámetros, se han ejecutado de nuevo los modelos y se ha calculado la métrica f1-score de los modelos. Los resultados han sido los siguientes:

**Arboles de decisión**

Modelo anterior:
- datos train: 0.5521910321268637
- datos test: 0.4662880217178749

Modelos grid:
- datos train: 0.5609680913335696
- datos test: 0.4738335374223451


**k-vecinos**

Modelo anterior:
- datos train: 0.47694603914593897
- datos test: 0.43776079991626043

Modelos grid:
- datos train: 0.4552436960177454
- datos test: 0.42664070107775254



**SVM**

Modelo anterior:
- datos train: 0.629242520791558
- datos test: 0.5306070173027698


Modelos grid:
- datos train: 0.6253405316778935
- datos test: 0.5798695111847556


**Logistic Regression**

Modelo anterior:
- datos train: 0.6716249668433196
- datos test: 0.6210797267076509


Modelos grid:
- datos train: 0.6624594010643858
- datos test: 0.6299135643129212



## Detección nivel de toxicidad

### Despliegue de modelos, evaluación con f1-score y CEM

In [10]:
y = df['toxicity_level']
X = tfidf
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

### CEM

In [ ]:
import numpy as np

from sklearn.metrics import f1_score
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold


def cross_validate(X, y, n_folds, model_class, random_seed=None, show_progress=False):
    model_name = model_class().__class__.__name__
    print('\n' + '#' * int(len(model_name) * 2))
    print(' ' * int(len(model_name) / 2) + model_name)
    print('#' * int(len(model_name) * 2) + '\n' * show_progress)

    kfold = StratifiedKFold(n_folds, shuffle=True, random_state=random_seed)
    fold = 1
    all_results = {'f1_toxic': [],
                   'cem': [],
                   'rbp': [],
                   'pearson': []}
    for trn_idxs, tst_idxs in kfold.split(X, y):
        if show_progress:
            print(f'+++ Processing fold {fold} / {n_folds}...')
        X_train, X_test = X[trn_idxs], X[tst_idxs]
        y_train, y_test = y[trn_idxs], y[tst_idxs]

        # Initialize and train the baseline model
        model = model_class()
        model.fit(X_train, y_train)

        # Evaluate the trained model
        results = model.score(X_test, y_test)
        fold += 1

        for key, val in results.items():
            all_results[key].append(val)

    print(f'\nRESULTS:\n\t'
          f'F1-Toxic: {np.mean(all_results["f1_toxic"]):.2f} \u00b1 {np.std(all_results["f1_toxic"]):.2f}\n\t'
          f'CEM: {np.mean(all_results["cem"]):.2f} \u00b1 {np.std(all_results["cem"]):.2f}\n\t'
          f'RBP: {np.mean(all_results["rbp"]):.2f} \u00b1 {np.std(all_results["rbp"]):.2f}\n\t'
          f'Pearson Coefficient: {np.mean(all_results["pearson"]):.2f} \u00b1 {np.std(all_results["pearson"]):.2f}')

    return all_results


class Evaluator:
    @staticmethod
    def _f1_toxic(y, y_pred, labels=(0, 1)):
        return f1_score(y, y_pred, labels=labels, average=None)[1]

    def _cem_score(self, y, y_pred):
        def _proximity(g_d, s_d, n_c):
            if g_d > s_d:
                return - np.log2((n_c[s_d] / 2 + np.sum(n_c[s_d + 1:g_d + 1])) / np.sum(n_c))
            elif g_d < s_d:
                return - np.log2((n_c[s_d] / 2 + np.sum(n_c[g_d:s_d])) / np.sum(n_c))
            else:
                return - np.log2(n_c[s_d] / 2 / np.sum(n_c))

        labels = [0, 1, 2, 3]
        gs_cs = [list(y).count(label) for label in labels]

        num = [_proximity(g, s, gs_cs) for g, s in zip(y, y_pred)]
        den = [_proximity(g, g, gs_cs) for g in y]
        return sum(num) / sum(den)

    @staticmethod
    def _rbp(y, y_pred):
        def get_relevance(label):
            # Relevance defined as the toxicity level normalized by the maximum relevance
            return (2 ** label - 1) / 2 ** labels[-1]

        def get_estimated_prob(label, counts):
            # Get the number of cases above the predicted class in a ranking (assuming an ordered list of label counts)
            offset = np.sum((counts + [0])[label + 1:])
            # Get all possible positions in a ranking for a possible label
            idxs = np.arange(offset, offset + counts[label])
            # If the label appears at least once in the predictions, compute the estimated probability of that class
            if len(idxs) > 0:
                return np.mean(p ** idxs)
            else:
                return 1

        def get_score(gs, pred):
            return (1 - p) * np.sum([relevances[y_i] * estimated_probs[y_pred_i] for y_i, y_pred_i in zip(gs, pred)])

        # Commonly adopted probability value
        p = 0.8

        # Obtain the estimated position per class given the predictions (Toxicity ranking with higher toxicity on top)
        labels = [0, 1, 2, 3]
        pred_cs = [list(y_pred).count(label) for label in labels]

        estimated_probs = [get_estimated_prob(label, pred_cs) for label in labels]
        relevances = [get_relevance(label) for label in labels]

        return get_score(y, y_pred)

    @staticmethod
    def _pearson_coef(y, y_pred):
        return pearsonr(y, y_pred)[0]

    @staticmethod
    def _check_inconsistencies(y, y_pred):
        first_inconsistency_msg = f'Number of predictions ({len(list(y_pred))}) do not match ' \
                                  f'the number of gold standard labels ({len(list(y))})'
        assert len(list(y)) == len(list(y_pred)), first_inconsistency_msg

        second_incosistency_msg = f'Prediction labels contain an invalid label ' \
                                  f'(valid_labels: [0, 1, 2, 3], predicted_labels: {list(set(y_pred))})'
        assert np.alltrue(np.isin(y_pred, [0, 1, 2, 3])), second_incosistency_msg

    def evaluate(self, y, y_pred):
        self._check_inconsistencies(y, y_pred)
        return {'f1_toxic': self._f1_toxic(np.array(y) > 0, np.array(y_pred) > 0),
                'cem': self._cem_score(y, y_pred),
                'rbp': self._rbp(y, y_pred),
                'pearson': self._pearson_coef(y, y_pred)}

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime


DATA_PATH = './data'
RESULTS_PATH = './results'


def read_processed_data(file):
    data = pd.read_csv(os.path.join(DATA_PATH, file), encoding='utf-8')
    return data


def read_results(gs_file, preds_file):
    def process_file(file):
        with open(os.path.join(RESULTS_PATH, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()

        idxs, results = zip(*[line.rstrip().split('\t') for line in lines])
        return np.array(results)[np.argsort(np.array(idxs).astype(int))].astype(int)

    return process_file(gs_file), process_file(preds_file)


def write_results(labels, filename):
    with open(os.path.join(RESULTS_PATH, filename), 'w') as f:
        for i, label in enumerate(labels):
            last = ((len(labels) - 1) == i)
            f.write(f'{i}\t{label}' + '\n' * (not last))


#### Árboles de desición

In [11]:
clf_dt = tree.DecisionTreeClassifier(random_state = 0, class_weight="balanced",
                                     max_depth = 6)
clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)

In [12]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_dt))
print(classification_report(y_test,y_pred_dt))

scores_dt = cross_val_score(clf_dt, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_dt.mean())
scores_dt = cross_val_score(clf_dt, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_dt.mean())

Accuracy del modelo: 0.6349206349206349
              precision    recall  f1-score   support

           0       0.67      0.95      0.78       449
           1       0.71      0.03      0.05       176
           2       0.28      0.13      0.17        55
           3       0.10      0.15      0.12        13

    accuracy                           0.63       693
   macro avg       0.44      0.31      0.28       693
weighted avg       0.64      0.63      0.54       693

Media de f1-score con cross_validation 0.236621245206792
Media de f1-score con cross_validation en test:  0.23769180596904832


In [ ]:
#CEM

cross_validate(X_train, y_train, model_class = tree.DecisionTreeClassifier, n_folds=5)


############################################
           DecisionTreeClassifier
############################################


KeyError: ignored

#### SVM

In [13]:
clf_svm = svm.SVC(random_state=0, class_weight="balanced")
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test)

In [14]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_svm))
print(classification_report(y_test,y_pred_svm))

scores_svm = cross_val_score(clf_svm, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_svm.mean())

scores_svm = cross_val_score(clf_svm, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_svm.mean())

Accuracy del modelo: 0.6724386724386724
              precision    recall  f1-score   support

           0       0.69      0.97      0.81       449
           1       0.48      0.16      0.25       176
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        13

    accuracy                           0.67       693
   macro avg       0.29      0.28      0.26       693
weighted avg       0.57      0.67      0.59       693



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Media de f1-score con cross_validation 0.2689509027093041
Media de f1-score con cross_validation en test:  0.22383528876396835


#### Logistic regression

In [15]:
clf_log = LogisticRegression(random_state = 0,class_weight="balanced")
clf_log.fit(X_train, y_train)
y_pred_log = clf_log.predict(X_test)

In [16]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_log))
print(classification_report(y_test,y_pred_log))
scores_log = cross_val_score(clf_log, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_log.mean())
scores_log = cross_val_score(clf_log, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_log.mean())

Accuracy del modelo: 0.6277056277056277
              precision    recall  f1-score   support

           0       0.74      0.83      0.78       449
           1       0.35      0.28      0.31       176
           2       0.29      0.22      0.25        55
           3       0.29      0.15      0.20        13

    accuracy                           0.63       693
   macro avg       0.42      0.37      0.39       693
weighted avg       0.59      0.63      0.61       693

Media de f1-score con cross_validation 0.36229624616326883
Media de f1-score con cross_validation en test:  0.2669652097024179


#### KNN

In [17]:
# Obtención del número óptimo de vecinos

neighbors = list(range(1,10,2)) 
cv_scores = [ ]

for K in neighbors: #(k=1,k=3,k=5,k=7 y k=9)
    knn = KNeighborsClassifier(n_neighbors = K,weights="distance",metric="euclidean")
    scores = cross_val_score(knn,X_train,y_train,cv = 5,scoring ="accuracy")
    cv_scores.append(scores.mean())

mse = [1-x for x in cv_scores]

optimal_k = neighbors[mse.index(min(mse))] #calculo el minimo error obtenido para k
print("The optimal no. of neighbors is {}".format(optimal_k))

The optimal no. of neighbors is 3


In [18]:
clf_knn = KNeighborsClassifier(n_neighbors = 3,weights="distance",metric="euclidean")
clf_knn.fit(X_train,y_train)
y_pred_knn = clf_knn.predict(X_test)

In [19]:
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_knn))
print(classification_report(y_test,y_pred_knn))
scores_knn = cross_val_score(clf_knn, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_knn.mean())
scores_knn = cross_val_score(clf_knn, X_test, y_test, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_knn.mean())

Accuracy del modelo: 0.6608946608946609
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       449
           1       0.85      0.06      0.12       176
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        13

    accuracy                           0.66       693
   macro avg       0.38      0.26      0.23       693
weighted avg       0.64      0.66      0.54       693

Media de f1-score con cross_validation 0.2389720501642763
Media de f1-score con cross_validation en test:  0.21911746383999442


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Grid search

#### Árboles de decisión

In [ ]:
model_dt = tree.DecisionTreeClassifier(class_weight='balanced',random_state = 0)

grid_dt=dict(criterion= ("gini", "entropy"),splitter=("best", "random"),
             max_depth=list(range(1,8)))

clf_dt = GridSearchCV(estimator = model_dt,
                     param_grid = grid_dt)

mod_dt=clf_dt.fit(X_train, y_train)
y_pred_dt = clf_dt.predict(X_test)
mod_dt.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=2, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=0, splitter='random')

In [20]:
#clf_searc_dt = tree.DecisionTreeClassifier(random_state = 0, class_weight="balanced",
 #                                    max_depth = 1, criterion = "gini",splitter="random")

clf_searc_dt = tree.DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=2, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=0, splitter='random')

clf_searc_dt.fit(X_train, y_train)
y_pred_searc_dt = clf_searc_dt.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test,y_pred_searc_dt))
scores_dt = cross_val_score(clf_searc_dt, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation: ", scores_dt.mean())

scores_dt_test = cross_val_score(clf_searc_dt, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation del test: ", scores_dt_test.mean())
print(classification_report(y_test,y_pred_searc_dt))

Accuracy del modelo: 0.6334776334776335
Media de f1-score con cross_validation:  0.26793186764905486
Media de f1-score con cross_validation del test:  0.1761305235498877
              precision    recall  f1-score   support

           0       0.67      0.96      0.79       449
           1       0.00      0.00      0.00       176
           2       0.00      0.00      0.00        55
           3       0.13      0.46      0.20        13

    accuracy                           0.63       693
   macro avg       0.20      0.36      0.25       693
weighted avg       0.44      0.63      0.52       693



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
adc_pred_cem = pd.DataFrame()
adc_pred_cem['predicho'] = y_pred_searc_dt


#### SVM

In [ ]:
clf_search_svm = GridSearchCV(estimator=svm.SVC(),
                              param_grid={'C': [1, 10],
                                          'kernel': ('linear', 'rbf','poly','sigmoid')
                                         })
m_svm = clf_search_svm.fit(X_train, y_train)
m_svm.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
#clf_linear_svm = svm.SVC(kernel='linear',C=1,class_weight="balanced")
clf_linear_svm = svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

clf_linear_svm.fit(X_train, y_train)
y_pred_linear_svm = clf_linear_svm.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test,y_pred_linear_svm))
print(classification_report(y_test,y_pred_linear_svm))

scores_knn = cross_val_score(clf_linear_svm, X_train, y_train, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation: ", scores_knn.mean())

scores_knn_test = cross_val_score(clf_linear_svm, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation del test: ", scores_knn_test.mean())

Accuracy del modelo: 0.6666666666666666
              precision    recall  f1-score   support

           0       0.69      0.98      0.81       449
           1       0.44      0.13      0.20       176
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        13

    accuracy                           0.67       693
   macro avg       0.28      0.28      0.25       693
weighted avg       0.56      0.67      0.57       693



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Media de f1-score con cross_validation:  0.2724043410286597
Media de f1-score con cross_validation del test:  0.23468291195028862


#### Logistic regression

In [ ]:
from scipy.stats import uniform, norm
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold


model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2','l1','elasticnet','none']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search

grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid,
                           n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor: 0.701444 usando {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}


In [22]:
clf_log_tun = LogisticRegression(random_state = 0, C=1,penalty='l1'
                                 ,solver = 'liblinear',class_weight="balanced")
clf_log_tun.fit(X_train, y_train)
y_pred_log_tun = clf_log_tun.predict(X_test)

In [23]:
scores_log_tun = cross_val_score(clf_log_tun, X_train, y_train, cv=5, scoring='f1_macro')
print("Accuracy del modelo:", accuracy_score(y_test,y_pred_log_tun))
print(classification_report(y_test,y_pred_log_tun))
print("Media de f1-score con cross_validation", scores_log_tun.mean())

scores_logistic_tun = cross_val_score(clf_log_tun, X_test, y_test, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_logistic_tun.mean())

Accuracy del modelo: 0.6681096681096681
              precision    recall  f1-score   support

           0       0.71      0.94      0.81       449
           1       0.48      0.14      0.21       176
           2       0.38      0.24      0.29        55
           3       0.14      0.15      0.15        13

    accuracy                           0.67       693
   macro avg       0.43      0.37      0.37       693
weighted avg       0.62      0.67      0.61       693

Media de f1-score con cross_validation 0.3300414693660893
Media de f1-score con cross_validation en test:  0.23027051021669856


#### KNN

In [24]:
from sklearn.model_selection import RepeatedStratifiedKFold
model = KNeighborsClassifier()
vecinos = list(range(1,10))
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

# define grid search
grid = dict(n_neighbors=vecinos,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, 
                           cv=cv, scoring='accuracy',error_score=0)

grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

KeyboardInterrupt: ignored

In [25]:
clf_searc_knn = KNeighborsClassifier(n_neighbors = 3,
                                     weights="uniform",metric="euclidean")

clf_searc_knn.fit(X_train,y_train)
y_pred_searc_knn = clf_searc_knn.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test,y_pred_searc_knn))
print(classification_report(y_test,y_pred_searc_knn))

scores_searc_knn = cross_val_score(clf_searc_knn, X_train, y_train, 
                                   cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_searc_knn.mean())

scores_searc_knn2 = cross_val_score(clf_searc_knn, X_test, y_test, 
                                    cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_searc_knn2.mean())

Accuracy del modelo: 0.6594516594516594
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       449
           1       0.83      0.06      0.11       176
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        13

    accuracy                           0.66       693
   macro avg       0.37      0.26      0.22       693
weighted avg       0.64      0.66      0.54       693

Media de f1-score con cross_validation 0.2375762253566564


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Media de f1-score con cross_validation en test:  0.20797193553395737


## Solución

En segundo lugar, se ha modelado no si los textos son tóxicos o no, sino qué nivel de toxicidad presentaba cada uno de ellos. Para ello, se le ha dado a los modelos una variable llamada toxicity_level, la cual mide del 0 al 3 cómo de tóxico es un texto:
1. 0 no tóxico
2. 1 levemente tóxico
3. 2 tóxico
4. 3 muy tóxico

La idea es la misma que en el caso anterior; a los modelos se le dan las tupas de palabras como entrada, pero el nivel de toxicidad como salida. Con los datos de entrenamiento se entrenan los mismos modelos que en caso anterior y con los datos de test se validan. Los resultados han sido los siguientes:

- Árboles de decisión: 0.5396825396825397
- SVM kernel por defecto: 0.6738816738816739
- SVM kernel lineal: 0.6378066378066378
- SVM kernel polinomial: 0.6608946608946609
- SVM kernel sigmoide: 0.6248196248196248
- Regresión Logística: 0.6378066378066378
- Redes Neuronales: 0.6479076479076479

Los resultados no son nada buenos ya que prácticamente los modelos escogen aleatoriamente a qué clase pertenece cada observación. Sin embargo, dentro de lo malo, el modelo que mejor precisión tiene es máquinas de vectores soporte con kernel por defecto (rbf) con una precisión del 67,3%

## Solución Grid



Se han realizado los mismos pasos que para los modelos de detección de toxicidad y estos han sido los resultados:

**Arboles de decisión**



Modelo anterior:



- datos train: 0.236621245206792
- datos test: 0.23769180596904832



Modelos grid:



- datos train: 0.26793186764905486
- datos test: 0.1761305235498877





**k-vecinos**



Modelo anterior:



- datos train: 0.2389720501642763
- datos test: 0.21911746383999442



Modelos grid:



- datos train: 0.2375762253566564
- datos test: 0.20797193553395737





**SVM**



Modelo anterior:



- datos train: 0.2689509027093041
- datos test: 0.22383528876396835





Modelos grid:



- datos train: 0.2724043410286597
- datos test: 0.23468291195028862






**Logistic Regression**



Modelo anterior:



- datos train: 0.36229624616326883
- datos test: 0.2669652097024179



Modelos grid:



- datos train: 00.3300414693660893
- datos test: 0.23027051021669856

Se puede observar que los resultados no son nada buenos e incluso luego de hacer el grid, empeoran. En la siguiente sesión abordaremos más modelos como el Bagging o Boosting para ver si hay alguna mejora

## Combinación de modelos

## Detección de toxicidad

In [ ]:
y_comb = df['toxicity']
X_comb = tfidf
X_train_comb, X_test_comb,y_train_comb, y_test_comb = train_test_split(X_comb,
                                              y_comb,
                                                test_size = 0.2, 
                                                random_state = 0)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
m_bagging = BaggingClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced"),
                               n_estimators=10)
m_bagging.fit(X_train_comb, y_train_comb)
m_pred_log = m_bagging.predict(X_test_comb)

In [ ]:
scores_comb_log = cross_val_score(m_bagging, X_train_comb, y_train_comb,
                                  cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_comb_log.mean())
scores_comb_log = cross_val_score(m_bagging, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_comb_log.mean())
print(classification_report(y_test,m_pred_log))

Media de f1-score con cross_validation 0.6516730907188738
Media de f1-score con cross_validation en test:  0.6074752812576284
              precision    recall  f1-score   support

           0       0.75      0.87      0.80       449
           1       0.65      0.46      0.54       244

    accuracy                           0.72       693
   macro avg       0.70      0.66      0.67       693
weighted avg       0.71      0.72      0.71       693



In [ ]:
m_bagging_dt = BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=2, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=0, splitter='random'), n_estimators=10)

m_bagging_dt.fit(X_train_comb, y_train_comb)
m_pred_dt = m_bagging_dt.predict(X_test_comb)

In [ ]:
scores_comb_dt = cross_val_score(m_bagging_dt, X_train_comb, y_train_comb,
                                  cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_comb_dt.mean())
scores_comb_dt = cross_val_score(m_bagging_dt, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_comb_dt.mean())
print(classification_report(y_test,m_pred_dt))

Media de f1-score con cross_validation 0.5672954257580707
Media de f1-score con cross_validation en test:  0.5276695920269814
              precision    recall  f1-score   support

           0       0.69      0.95      0.80       449
           1       0.68      0.20      0.31       244

    accuracy                           0.69       693
   macro avg       0.68      0.57      0.55       693
weighted avg       0.68      0.69      0.63       693



### Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
boosting = AdaBoostClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
,solver = 'liblinear',class_weight="balanced"),n_estimators=100)
boosting.fit(X_train_comb, y_train_comb)
y_pred_boost = boosting.predict(X_test_comb)

In [ ]:
cross_boosting = cross_val_score(boosting, X_train_comb, y_train_comb, cv=5, scoring='f1_macro')
print("Accuracy del modelo:", accuracy_score(y_test_comb,y_pred_boost))
print(classification_report(y_test_comb,y_pred_boost))
print("Media de f1-score con cross_validation", cross_boosting.mean())

scores_boosting = cross_val_score(boosting, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_boosting.mean())

Accuracy del modelo: 0.7085137085137085
              precision    recall  f1-score   support

           0       0.76      0.81      0.78       449
           1       0.60      0.52      0.56       244

    accuracy                           0.71       693
   macro avg       0.68      0.67      0.67       693
weighted avg       0.70      0.71      0.70       693

Media de f1-score con cross_validation 0.6613159407504455
Media de f1-score con cross_validation en test:  0.6292702645073429


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
m_rf = RandomForestClassifier(n_estimators=10, max_depth=6,random_state=0)
m_rf.fit(X_train_comb, y_train_comb)
y_pred_rf = m_rf.predict(X_test_comb)

cross_rf = cross_val_score(m_rf, X_train_comb, y_train_comb, cv=5, scoring='f1_macro')
print(classification_report(y_test_comb,y_pred_rf))
print("Media de f1-score con cross_validation", cross_rf.mean())

scores_rf = cross_val_score(m_rf, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_rf.mean())

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       449
           1       1.00      0.00      0.01       244

    accuracy                           0.65       693
   macro avg       0.82      0.50      0.40       693
weighted avg       0.77      0.65      0.51       693

Media de f1-score con cross_validation 0.41077932757271257
Media de f1-score con cross_validation en test:  0.40018361993288554


In [ ]:
###

### XGBoost

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
model_xg = XGBClassifier()
model_xg.fit(X_train_comb, y_train_comb)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
pred_xg = model_xg.predict(X_test_comb)

In [ ]:
cross_xg = cross_val_score(model_xg, X_train_comb, y_train_comb, cv=5, scoring='f1_macro')
print(classification_report(y_test_comb,pred_xg))
print("Media de f1-score con cross_validation", cross_xg.mean())
scores_xg = cross_val_score(model_xg, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation en test: ", scores_xg.mean())


              precision    recall  f1-score   support

           0       0.70      0.96      0.81       449
           1       0.79      0.26      0.39       244

    accuracy                           0.71       693
   macro avg       0.75      0.61      0.60       693
weighted avg       0.73      0.71      0.66       693

Media de f1-score con cross_validation 0.5950328195731487
Media de f1-score con cross_validation en test:  0.5037391403101498


### Stacking

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

estimators = [('mlp', MLPClassifier(random_state=1, max_iter=300)),('svr', svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

stacking = StackingClassifier(estimators=estimators, final_estimator=tree.DecisionTreeClassifier())
stacking.fit(X_train_comb, y_train_comb)
stack = stacking.predict(X_test_comb)

In [ ]:
cross_stack = cross_val_score(stacking, X_train_comb, y_train_comb, cv=5, scoring='f1_macro')


scores_stack = cross_val_score(stacking, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')
print(classification_report(y_test_comb,stack))
print("Media de f1-score con cross_validation", cross_stack.mean())
print("Media de f1-score con cross_validation en test: ", scores_stack.mean())

              precision    recall  f1-score   support

           0       0.71      0.72      0.71       449
           1       0.47      0.47      0.47       244

    accuracy                           0.63       693
   macro avg       0.59      0.59      0.59       693
weighted avg       0.63      0.63      0.63       693

Media de f1-score con cross_validation 0.589853128123402
Media de f1-score con cross_validation en test:  0.5572715983397247


## Solución modelos combinados

Para esta tarea se han modelado nuestros datos con ensemble methods y modelos híbridos. Concretamente:



Emsemble Methods:



- Bagging

- Boosting

- Random Forests

- XGBoost

Modelos Híbridos:

- Stacking 


Luego de aplicar diversos modelos de ML para la tarea de detección de toxicidad, se han obtenido los siguientes resultados de f1-score con CV de 5 particiones:

**Bagging**

- Modelo con datos train: 0.6516730907188738

- Modelo con datos test: 0.6074752812576284


**Boosting**

- Modelo con datos train: 0.6613159407504455

- Modelo con datos test: 0.6292702645073429


**Random Forest**

- Modelo con datos train: 0.41077932757271257

- Modelo con datos test: 0.40018361993288554


**XGBoost**

- Modelo con datos train: 0.5950328195731487

- Modelo con datos test: 0.5037391403101498


**Stacking**

- Modelo con datos train: 0.589853128123402

- Modelo con datos test: 0.5572715983397247


En este caso, Boosting es el modelo que mejores métricas de f1-score proporciona, con un valor aproximado a 0.63 para los datos de test.


## Niveles de toxicidad

In [ ]:
y_level = df['toxicity_level']
X_level = tfidf
X_train_level, X_test_level, y_train_level, y_test_level = train_test_split(X_level,y_level,test_size=0.2, random_state=0)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
m_bagging = BaggingClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced"),
                               n_estimators=10)
m_bagging.fit(X_train_level, y_train_level)
m_pred_log = m_bagging.predict(X_test_level)

In [ ]:
scores_level_log = cross_val_score(m_bagging, X_train_level, y_train_level,
                                  cv=5, scoring='f1_macro')
print("Media de f1-score con cross_validation", scores_level_log.mean())
scores_comb_log = cross_val_score(m_bagging, X_test_level, y_test_level, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_level_log.mean())
print(classification_report(y_test_level,m_pred_log))

Media de f1-score con cross_validation 0.31579558843432787
Media de f1-score con cross_validation en test:  0.31579558843432787
              precision    recall  f1-score   support

           0       0.70      0.93      0.80       449
           1       0.41      0.21      0.28       176
           2       0.25      0.04      0.06        55
           3       0.00      0.00      0.00        13

    accuracy                           0.66       693
   macro avg       0.34      0.29      0.29       693
weighted avg       0.58      0.66      0.59       693



### Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
boosting = AdaBoostClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
,solver = 'liblinear',class_weight="balanced"),n_estimators=100)
boosting.fit(X_train_level, y_train_level)
y_pred_boost = boosting.predict(X_test_level)

In [ ]:
cross_boosting = cross_val_score(boosting, X_train_level, y_train_level, cv=5, scoring='f1_macro')
print("Accuracy del modelo:", accuracy_score(y_test_level,y_pred_boost))
print(classification_report(y_test_level,y_pred_boost))
print("Media de f1-score con cross_validation", cross_boosting.mean())

scores_boosting = cross_val_score(boosting, X_test_level, y_test_level, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_boosting.mean())

Accuracy del modelo: 0.658008658008658
              precision    recall  f1-score   support

           0       0.67      0.97      0.79       449
           1       0.49      0.10      0.17       176
           2       0.20      0.02      0.03        55
           3       0.00      0.00      0.00        13

    accuracy                           0.66       693
   macro avg       0.34      0.27      0.25       693
weighted avg       0.57      0.66      0.56       693

Media de f1-score con cross_validation 0.28525106187767724


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Media de f1-score con cross_validation en test:  0.2317631766119695


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
m_rf = RandomForestClassifier(n_estimators=10, max_depth=6,random_state=0)
m_rf.fit(X_train_level, y_train_level)
y_pred_rf = m_rf.predict(X_test_level)

cross_rf = cross_val_score(m_rf, X_train_level, y_train_level, cv=5, scoring='f1_macro')
print(classification_report(y_test_level,y_pred_rf))
print("Media de f1-score con cross_validation", cross_rf.mean())

scores_rf = cross_val_score(m_rf, X_test_level, y_test_level, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_rf.mean())

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       449
           1       0.00      0.00      0.00       176
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        13

    accuracy                           0.65       693
   macro avg       0.16      0.25      0.20       693
weighted avg       0.42      0.65      0.51       693

Media de f1-score con cross_validation 0.20142329725105088
Media de f1-score con cross_validation en test:  0.19631885390331724


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### XGBoost

In [ ]:
model_xg = XGBClassifier()
model_xg.fit(X_train_level, y_train_level)
pred_xg = model_xg.predict(X_test_level)

In [ ]:
cross_xg = cross_val_score(model_xg, X_train_level, y_train_level, cv=5, scoring='f1_macro')
print(classification_report(y_test_level,pred_xg))
print("Media de f1-score con cross_validation", cross_xg.mean())

scores_xg = cross_val_score(model_xg, X_test_level, y_test_level, cv=5, scoring='f1_macro')

print("Media de f1-score con cross_validation en test: ", scores_xg.mean())

              precision    recall  f1-score   support

           0       0.68      0.96      0.80       449
           1       0.44      0.14      0.21       176
           2       0.33      0.04      0.07        55
           3       0.50      0.08      0.13        13

    accuracy                           0.66       693
   macro avg       0.49      0.30      0.30       693
weighted avg       0.59      0.66      0.58       693

Media de f1-score con cross_validation 0.3034461986954384
Media de f1-score con cross_validation en test:  0.2743515596253923


### Stacking

In [ ]:
estimators = [('mlp', MLPClassifier(random_state=1, max_iter=300)),('svr', svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

stacking = StackingClassifier(estimators=estimators, final_estimator=tree.DecisionTreeClassifier())
stacking.fit(X_train_comb, y_train_comb)
stack = stacking.predict(X_test_comb)

In [ ]:
cross_stack = cross_val_score(stacking, X_train_comb, y_train_comb, cv=5, scoring='f1_macro')


scores_stack = cross_val_score(stacking, X_test_comb, y_test_comb, cv=5, scoring='f1_macro')
print(classification_report(y_test_comb,stack))
print("Media de f1-score con cross_validation", cross_stack.mean())
print("Media de f1-score con cross_validation en test: ", scores_stack.mean())

              precision    recall  f1-score   support

           0       0.73      0.73      0.73       449
           1       0.50      0.49      0.50       244

    accuracy                           0.65       693
   macro avg       0.61      0.61      0.61       693
weighted avg       0.65      0.65      0.65       693

Media de f1-score con cross_validation 0.5830567867769515
Media de f1-score con cross_validation en test:  0.5485262356544693


## Solución modelos combinados

Realizando los mismos modelos y validaciones que en detección de toxicidad. Para los niveles de toxicidad se ha obtenido lo siguiente:

**Bagging**

- Modelo con datos train: 0.31579558843432787

- Modelo con datos test: 0.31579558843432787


**Boosting**

- Modelo con datos train: 0.28525106187767724

- Modelo con datos test: 0.2317631766119695


**Random Forest**

- Modelo con datos train: 0.20142329725105088

- Modelo con datos test: 0.19631885390331724


**XGBoost**

- Modelo con datos train: 0.3034461986954384

- Modelo con datos test: 0.2743515596253923


**Stacking**

- Modelo con datos train: 0.5830567867769515

- Modelo con datos test: 0.5485262356544693



Se puede observar que los mejores resultados los da Stacking Sin embargo, el f1-Score a penas llega al 0.6 por lo que el modelo no hace una buena clasificación. Habrá que ir probando otros modelos.